## To begin, let's establish a general order of operations for this project:

### - Research good parameter values for a Tausworthe generator (non-code work)

### - Implement the generator.

In [1]:
def tausworthe(seed = 123456789101112, quantity = 100):
    assert type(seed) == int, 'Your seed must be an integer!'
    assert len(str(seed)) >= 15, 'Your seed must be at least 15 characters long!'
    
    seed = int('1' + str(seed)) # final seed is at least 16 characters long, or at least 50 bits
    print('seed: ', seed)
    # print('binary representation of your seed: ', bin(seed))
    gen_input = bin(seed)[2:]
    
    # Establish parameters for this version of the generator
    r = 5
    q = 50
    
    #Generate the specified number of psuedorandom bits
    gen_output = gen_input
    for bit in range(len(gen_input), len(gen_input) + quantity + 1):
        
        if gen_output[bit - r] == gen_output[bit - q]:
            gen_output = gen_output + '0'
        else:
            gen_output = gen_output + '1'
            
    gen_output = gen_output[51:]
    print('generated bits: ', len(gen_output))
    
    return gen_output
    

In [2]:
def create_prns(bits, L = 5):
    # Use the bits generated above to generate Uniform(0,1) PRNs.
    
    # Split the input into groups of L bits.
    num_groups = len(bits) // L
    groups = [bits[i : i + L] for i in range(num_groups)]
    
    # Convert back to decimal; divide each group by 2 ** L to get a PRN
    PRNs = [int(group, 2) / 2 ** L for group in groups]
    
    return PRNs
    

### - Code some general tests.
#### Runs for independence, Chi-sq. for goodness of fit

##### Let's start with a chi-square test for GOF. Let's use four bins of size 0.25 each.

In [3]:
PRNs = create_prns(tausworthe(quantity = 400))
print('\n PRNS: \n', PRNs)

bins = {0.25: [], 0.5: [], 0.75: [], 1.0: []}
for interval in bins:
    for PRN in PRNs:
        if PRN <= interval and PRN > (interval - 0.25):
            bins[interval].append(PRN)
# print('\n Results: \n', bins)


seed:  1123456789101112
generated bits:  400

 PRNS: 
 [0.46875, 0.96875, 0.9375, 0.90625, 0.8125, 0.625, 0.25, 0.5, 0.03125, 0.0625, 0.15625, 0.34375, 0.71875, 0.46875, 0.9375, 0.875, 0.78125, 0.5625, 0.15625, 0.34375, 0.6875, 0.375, 0.78125, 0.59375, 0.1875, 0.375, 0.78125, 0.59375, 0.1875, 0.40625, 0.8125, 0.625, 0.25, 0.53125, 0.09375, 0.21875, 0.4375, 0.875, 0.78125, 0.5625, 0.125, 0.28125, 0.5625, 0.15625, 0.3125, 0.625, 0.28125, 0.59375, 0.1875, 0.40625, 0.84375, 0.6875, 0.40625, 0.84375, 0.71875, 0.46875, 0.9375, 0.90625, 0.8125, 0.65625, 0.3125, 0.65625, 0.3125, 0.65625, 0.34375, 0.6875, 0.375, 0.75, 0.5, 0.0, 0.0, 0.0, 0.03125, 0.09375, 0.1875, 0.375, 0.78125, 0.5625, 0.15625, 0.34375]


In [4]:
n = len(PRNs)
k = len(bins.keys())

Ei = n // k

Oi = [len(bins[bin]) for bin in bins]

chisqs = [(item - Ei) ** 2 / Ei for item in Oi]

print('Chi-sq. statistic: ', sum(chisqs))


Chi-sq. statistic:  0.8500000000000001


After referencing the chi-squared table, chisq(0.05, 3) = 7.81. Our statistic is lower than that, so we fail to reject the null hypothesis that our PRNs are Uniforms.

##### Let's try a runs above / below mean test for independence!

In [5]:
runs = [0] * len(PRNs)
for i, PRN in enumerate(PRNs):
    if PRN > 0.5:
        runs[i] = '+'
    else:
        runs[i] = '-'

print('runs: ', runs)

run_count = 1

for i in range(1, len(runs)):
    if runs[i - 1] != runs[i]:
        run_count += 1

print('\nnumber of runs: ', run_count)


runs:  ['-', '+', '+', '+', '+', '+', '-', '-', '-', '-', '-', '-', '+', '-', '+', '+', '+', '+', '-', '-', '+', '-', '+', '+', '-', '-', '+', '+', '-', '-', '+', '+', '-', '+', '-', '-', '-', '+', '+', '+', '-', '-', '+', '-', '-', '+', '-', '+', '-', '-', '+', '+', '-', '+', '+', '-', '+', '+', '+', '+', '-', '+', '-', '+', '-', '+', '-', '+', '-', '-', '-', '-', '-', '-', '-', '-', '+', '+', '-', '-']

number of runs:  41


In [6]:
B = run_count

n = len(PRNs)
n1 = len([i for i in PRNs if i > 0.5])
n2 = n - n1

print('n: ', n, '\nn1: ', n1, '\nn2: ', n2)

EB = 2 * n1 * n2 / n + 0.5
print('\nExpected number of runs: ', EB)

var_part_1 = 2 * n1 * n2
var_B = var_part_1 * (var_part_1 - n) / ((n ** 2) * (n - 1))

import math
print('\nStandard dev. of runs: ', math.sqrt(var_B))


Z = (B - EB) / (math.sqrt(var_B))

print('\nZ-statistic: ', Z)

n:  80 
n1:  38 
n2:  42

Expected number of runs:  40.4

Standard dev. of runs:  4.432489371145345

Z-statistic:  0.1353641147807113


per the tables, the relevant Z-statistic Z(0.025) is roughly 0.51. Therefore, we fail to reject the null hypothesis that these PRNs are independent.

### - Plot adjacent PRNs on a unit square! Sounds like a job for a loop.

### - Generate a few Nor(0, 1) RVs using the Tausworthe Uniforms.
#### Inverse Transform, or something else?